<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/c670_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 438, done.
remote: Counting objects: 100% (438/438), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 1620 (delta 274), reused 337 (delta 218), pack-reused 1182
Receiving objects: 100% (1620/1620), 38.32 MiB | 34.21 MiB/s, done.
Resolving deltas: 100% (1052/1052), done.


In [2]:
from c620.F import F
from c620.config import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


In [3]:
f = F(config)
f.Recommended_mode = False
f.real_data_mode = False
f.c670_model.x_col

['Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Hydrogen_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methane_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethane_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Propane_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Butane_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Pentane_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Hexane_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Benzene_wt%',
 'Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detai

# load raw data

In [5]:
url = '/content/drive/MyDrive/台塑輕油案子/data/c620/raw/Aroma-2 C620+C660+C670 Plant Operation Matching Results_Actual Tray_0407.xlsx'
c670_feed = pd.read_excel(url,sheet_name='C670 Feed').T.reset_index(drop=True).T
c670_feed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,2020 Jan \nOperation of C670,Date,Feed from C620 Btms,NaN,Feed from C660 Btms,NaN,NaN,2020 Feb\nOperation of C670,Date,Feed from C620 Btms,NaN,Feed from C660 Btms,NaN
1,NaN,NaN,Unit,Plant\nData,Simulation\nFeed,Plant\nData,Simulation\nFeed,NaN,NaN,Unit,Plant\nData,Simulation\nFeed,Plant\nData,Simulation\nFeed
2,NaN,Feed Rate,m3/hr,164.628,169.468,70.2381,61.4078,NaN,Feed Rate,m3/hr,144.645,143.98,44.3984,40.9152
3,NaN,NaN,ton/hr,143.749,147.735,未測量密度,53.4537,NaN,NaN,ton/hr,126.291,125.529,未測量密度,35.6099
4,NaN,H2,wt%,0,0,0,0,NaN,H2,wt%,0,0,0,0
5,NaN,METHANE,wt%,0,0,0,0,NaN,METHANE,wt%,0,0,0,0
6,NaN,ETHANE,wt%,0,0,0,0,NaN,ETHANE,wt%,0,0,0,0
7,NaN,PROPANE,wt%,0,0,0,0,NaN,PROPANE,wt%,0,0,0,0
8,NaN,BUTANE,wt%,3.2423e-24,4.30891e-24,3.80199e-27,3.93785e-27,NaN,BUTANE,wt%,5.57549e-24,6.809e-24,4.60916e-27,5.0372e-28
9,NaN,PENTANE,wt%,1.00672e-16,1.3379e-16,1.97033e-19,2.04074e-19,NaN,PENTANE,wt%,1.73117e-16,2.61502e-16,2.38863e-19,3.24345e-20


# 計算c670_combined_feed_wt1

In [9]:
Feed_from_C620_Btms_ton = 147.735
Feed_from_C660_Btms_ton = 53.4537
C620_Btms_wt = c670_feed.iloc[4:4+41,4]
C660_Btms_wt = c670_feed.iloc[4:4+41,6]
total_ton = Feed_from_C620_Btms_ton + Feed_from_C660_Btms_ton
Feed_from_C620_Btms_ton_p = Feed_from_C620_Btms_ton/total_ton 
Feed_from_C660_Btms_ton_p = Feed_from_C660_Btms_ton/total_ton
c670_combined_feed_wt = (C620_Btms_wt*Feed_from_C620_Btms_ton_p)+(C660_Btms_wt*Feed_from_C660_Btms_ton_p)
c670_combined_feed_wt = c670_combined_feed_wt.to_frame().T
c670_combined_feed_wt.columns = f.c670_model.x_col[:41]
c670_combined_feed_wt

,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Hydrogen_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Propane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Butane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Pentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Hexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Benzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Cyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Heptane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Water_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Toluene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Octane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclopentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_p-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_m-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_o-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Nonane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_i-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-3-ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,3,5-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-2-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,4-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_tert-Butylcyclohexane_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,3-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Indane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-n-propylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2-Diethylbenzene_wt%",Tol

# 計算c670_upper_bf1

In [26]:
import numpy as np
a = C660_Btms_wt * Feed_from_C660_Btms_ton
b = C620_Btms_wt * Feed_from_C620_Btms_ton
c = a+b
upper_bf = a / (c+1e-8)
upper_bf = upper_bf.to_frame().T
upper_bf = upper_bf.clip(lower=0.0,upper=1.0)
upper_bf.columns = f.c670_model.x_col[-41:]
upper_bf[list(set(f.index_9999)&set(upper_bf.columns))] = 0.9999
upper_bf[list(set(f.index_0001)&set(upper_bf.columns))] = 0.0001
upper_bf

,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Hydrogen_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Methane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Ethane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Propane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Butane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Pentane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Hexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Benzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Cyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Heptane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Water_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Methylcyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Toluene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Octane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Propylcyclopentane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Ethylcyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Ethylbenzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_p-Xylene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_m-Xylene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_o-Xylene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Nonane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_i-Propylbenzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Propylcyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Propylbenzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_1-Methyl-3-ethylbenzene_Fraction,Tolue

# 製作 c670_input dataframe1

In [27]:
c670_input1 = c670_combined_feed_wt.join(upper_bf)
c670_input1

,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Hydrogen_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Propane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Butane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Pentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Hexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Benzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Cyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Heptane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Water_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Toluene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Octane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclopentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_p-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_m-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_o-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Nonane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_i-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-3-ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,3,5-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-2-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,4-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_tert-Butylcyclohexane_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,3-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Indane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-n-propylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2-Diethylbenzene_wt%",Tol

In [29]:
op_hat = f.c670_model.predict(c670_input1).iloc[:,41*2:]
op_hat

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,0.87095,0.872197,206.455093,24.625813,24.482456,180.953568,224.384644


In [30]:
op_real = op_hat.copy()
op_real[:] = np.nan
op_real['Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr'] = 241.26
op_real['Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC'] = 181.5675964
op_real['Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC'] = 224
op_real

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,NaN,NaN,241.26,NaN,NaN,181.567596,224


In [31]:
op_col = op_real.dropna(axis=1).columns.tolist()
op_hat = op_hat[op_col]
op_real = op_real[op_col]

In [32]:
op_hat

,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,206.455093,180.953568,224.384644


In [33]:
op_real

,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,241.26,181.567596,224


In [35]:
diff = op_hat - op_real
diff

,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,-34.804907,-0.614029,0.384644


# 計算c670_combined_feed_wt2

In [36]:
Feed_from_C620_Btms_ton = 125.529
Feed_from_C660_Btms_ton = 35.6099
C620_Btms_wt = c670_feed.iloc[4:4+41,11]
C660_Btms_wt = c670_feed.iloc[4:4+41,13]
total_ton = Feed_from_C620_Btms_ton + Feed_from_C660_Btms_ton
Feed_from_C620_Btms_ton_p = Feed_from_C620_Btms_ton/total_ton 
Feed_from_C660_Btms_ton_p = Feed_from_C660_Btms_ton/total_ton
c670_combined_feed_wt = (C620_Btms_wt*Feed_from_C620_Btms_ton_p)+(C660_Btms_wt*Feed_from_C660_Btms_ton_p)
c670_combined_feed_wt = c670_combined_feed_wt.to_frame().T
c670_combined_feed_wt.columns = f.c670_model.x_col[:41]
c670_combined_feed_wt

,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Hydrogen_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Propane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Butane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Pentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Hexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Benzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Cyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Heptane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Water_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Toluene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Octane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclopentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_p-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_m-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_o-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Nonane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_i-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-3-ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,3,5-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-2-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,4-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_tert-Butylcyclohexane_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,3-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Indane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-n-propylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2-Diethylbenzene_wt%",Tol

# 計算c670_upper_bf2

In [37]:
import numpy as np
a = C660_Btms_wt * Feed_from_C660_Btms_ton
b = C620_Btms_wt * Feed_from_C620_Btms_ton
c = a+b
upper_bf = a / (c+1e-8)
upper_bf = upper_bf.to_frame().T
upper_bf = upper_bf.clip(lower=0.0,upper=1.0)
upper_bf.columns = f.c670_model.x_col[-41:]
upper_bf[list(set(f.index_9999)&set(upper_bf.columns))] = 0.9999
upper_bf[list(set(f.index_0001)&set(upper_bf.columns))] = 0.0001
upper_bf

,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Hydrogen_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Methane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Ethane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Propane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Butane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Pentane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Hexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Benzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Cyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Heptane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Water_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Methylcyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Toluene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Octane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Propylcyclopentane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Ethylcyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_Ethylbenzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_p-Xylene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_m-Xylene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_o-Xylene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Nonane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_i-Propylbenzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Propylcyclohexane_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_n-Propylbenzene_Fraction,Toluene Column C670 Feed Blending Factors Calculation_Upper Feed Blending Factor @ Tray 43 for Individual Component to Toluene Column C670_1-Methyl-3-ethylbenzene_Fraction,Tolue

# 製作 c670_input dataframe2

In [38]:
c670_input2 = c670_combined_feed_wt.join(upper_bf)
c670_input2

,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Hydrogen_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Propane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Butane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Pentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Hexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Benzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Cyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Heptane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Water_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Methylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Toluene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Octane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclopentane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_p-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_m-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_o-Xylene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Nonane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_i-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylcyclohexane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_n-Propylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-3-ethylbenzene_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,3,5-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-2-ethylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,4-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_tert-Butylcyclohexane_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2,3-Trimethylbenzene_wt%",Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_Indane_wt%,Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1-Methyl-4-n-propylbenzene_wt%,"Toluene Column C670 Operation_Combined Feed to Toluene Column C670 : Detailed weight % Composition_1,2-Diethylbenzene_wt%",Tol

In [39]:
op_hat = f.c670_model.predict(c670_input2).iloc[:,41*2:]
op_hat

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,0.87093,0.872209,249.171799,27.716675,27.958334,181.290924,224.975586


In [40]:
op_real = op_hat.copy()
op_real[:] = np.nan
op_real['Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr'] = 228.411499
op_real['Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC'] = 181.2036438
op_real['Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC'] = 224.1
op_real

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,NaN,NaN,228.411499,NaN,NaN,181.203644,224.1


In [41]:
op_col = op_real.dropna(axis=1).columns.tolist()
op_hat = op_hat[op_col]
op_real = op_real[op_col]

In [42]:
op_hat

,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,249.171799,181.290924,224.975586


In [43]:
op_real

,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,228.411499,181.203644,224.1


In [44]:
diff = op_hat - op_real
diff

,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
0,20.7603,0.08728,0.875586
